#### Monday, December 11, 2023

Going to add more samples to this notebook in an attempt to understand what does LangChain bring to the table ...

#### Thursday, December 7, 2023

[Intel Neural Chat 7B - Mistral meets new hardware & new data](https://www.youtube.com/watch?v=curhIJyuhYA)

https://colab.research.google.com/drive/14KvD_bMnXwLBqeKwaNZoWW4RqWt5kVBA?usp=sharing

I was able to step through this entire notebook. Nice!

In [ ]:
#  !pip -q install git+https://github.com/huggingface/transformers # often need to install from github
# !pip install -q datasets loralib sentencepiece
# !pip -q install bitsandbytes accelerate xformers einops
# !pip -q install langchain
# !pip install -q -U git+https://github.com/huggingface/peft.git
# # pip install -q -U git+https://github.com/huggingface/accelerate.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0

In [1]:
!nvidia-smi

Mon Dec 11 15:23:41 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        On  | 00000000:01:00.0  On |                  N/A |
|  0%   58C    P0              N/A /  70W |    487MiB /  2048MiB |     31%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Intel/neural-chat-7b-v3-1



In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
# running this does not seem to do anything ... the model does NOT load to GPU ...
torch.set_default_device('cuda')

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
model_name = 'Intel/neural-chat-7b-v3-1'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
                                          

In [7]:
# running this immediately sucks up ALL 4090 VRAM and fails to load this model.
# ... it does not chug away attempting to load the model.
# model = transformers.AutoModelForCausalLM.from_pretrained(model_name)

# This chugs through trying to load it, but it too sucks up all 4090 VRAM and fails to load the model
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=False, revision="main")

# This works! Use it! 
# this loads it to the GPU 8252 MiB VRAM
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, torch_dtype=torch.float16)

# 15.0s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
!nvidia-smi

Mon Dec 11 15:24:32 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        On  | 00000000:01:00.0  On |                  N/A |
|  0%   59C    P0              N/A /  70W |    515MiB /  2048MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

# encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

# device = 'cuda'
# model_inputs = encodeds.to(device)
# # model.to(device)

# generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

In [ ]:
# pipe =

### Prompt Format
```
### System:
{system}
### User:
{usr}
### Assistant:

```

In [9]:
system_prompt = "You are a helpful assistant"
input_text = "Let me tell you an interesting story about cat Tom and mouse Jerry"

In [10]:
# Format the input using the provided template
prompt = f"### System:\n{system_prompt}\n### User:\n{input_text}\n### Assistant:\n"
prompt

'### System:\nYou are a helpful assistant\n### User:\nLet me tell you an interesting story about cat Tom and mouse Jerry\n### Assistant:\n'

In [11]:
# Running this does not add anything to VRAM
# Tokenize and encode the prompt
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(device)

In [12]:
max_length = 512
# Generate a response
outputs = model.generate(inputs,
                            max_new_tokens=max_length,
                            pad_token_id=2,
                            num_return_sequences=1)

In [13]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [14]:
# Extract only the assistant's response
text = response.split("### Assistant:\n")[-1]

In [16]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [17]:
wrapped_text = wrap_text(text)
print(wrapped_text)

 Once upon a time, there lived a mischievous mouse named Jerry and a clever cat named Tom.
They both resided in a cozy little house, sharing the same space but never getting along.

Jerry was a quick-witted mouse who loved to play pranks on Tom. He would often hide in
Tom's favorite spots, such as his food bowl or under his bed, waiting for the perfect
moment to surprise him. Tom, on the other hand, was a cunning cat who was always on the
lookout for Jerry, hoping to catch him and put an end to his antics.

One day, Jerry decided to take his pranking to a whole new level. He found a box of
colorful crayons and began to draw a detailed map on the floor, leading Tom on a wild
goose chase. The map had twists and turns, leading Tom through various rooms and even
outside the house.

Tom, determined to catch Jerry, followed the map diligently, only to find himself lost in
a maze of his own making. Meanwhile, Jerry was hiding in a secret spot, watching Tom's
hilarious attempts to find him.

A

In [18]:
# def generate(input_text, system_prompt="",max_length=512):
#     # prompt = f"""<s>[INST]{input_text}[/INST]"""
#     prompt = f"""{input_text}"""
#     inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
#     outputs = model.generate(**inputs,
#                              max_new_tokens=max_length,
#                              temperature=0.1,
#                              do_sample=True)
#     text = tokenizer.batch_decode(outputs)[0]
#     wrapped_text = wrap_text(text)
#     print(wrapped_text)

def generate(input_text, system_prompt="You are a helpful assistant",max_length=512):

    # Format the input using the provided template
    prompt = f"### System:\n{system_prompt}\n### User:\n{input_text}\n### Assistant:\n"

    # Tokenize and encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(device)

    # Generate a response
    outputs = model.generate(inputs,
                             max_new_tokens=max_length,
                             pad_token_id=2,
                             num_return_sequences=1)
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)


    # Extract only the assistant's response
    text = response.split("### Assistant:\n")[-1]
    
    wrapped_text = wrap_text(text)
    print(wrapped_text)


## Instruction Answering

In [19]:
generate('Let me tell you an interesting story about cat Tom and mouse Jerry,',
         max_length=512)

# 2m 10.5s

 Once upon a time, in a cozy little house, there lived two very different creatures: a
clever and mischievous cat named Tom, and a quick-witted and resourceful mouse named
Jerry. These two characters were always at odds with each other, constantly engaged in a
battle of wits and cunning.

Tom, the cat, was a lazy and somewhat clumsy feline who loved nothing more than a good nap
in the sun. He was known for his love of food and his ability to find it, no matter where
it was hidden. However, Tom's laziness often led him to underestimate his opponent, Jerry,
the mouse.

Jerry, on the other hand, was a small but mighty mouse who was always on the lookout for
danger. He was incredibly agile and could move through the most confined spaces with ease.
Jerry was also quite clever, using his wits to outsmart Tom and protect his home from the
cat's relentless pursuit.

Despite their constant rivalry, Tom and Jerry shared a unique bond. They both had a deep
love for their home and the comfort it p

In [20]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

# 32.5s

 Mathematics, like a lighthouse, serves as a beacon of knowledge and guidance in the vast
expanse of human understanding. Both are essential structures that have played significant
roles in shaping our worldview and navigating through life's complexities.

Mathematics, as a discipline, is a systematic study of numbers, shapes, and patterns that
underpin various aspects of our lives. It is a language that allows us to communicate and
solve problems in a logical and structured manner. Similar to a lighthouse, mathematics
provides a stable foundation and a reliable source of guidance in the ever-changing world.



In [13]:
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

         # 1m 2.8s

 Mathematics and music can be compared as two distinct yet interconnected artistic
disciplines that share several similarities in their structure, principles, and the way
they evoke emotions and stimulate the human mind.

1. Structure: Both mathematics and music have a well-defined structure that follows
specific rules and patterns. In mathematics, these rules are often represented by
formulas, equations, and theorems, while in music, they are expressed through notes,
scales, and chords. These structures provide a framework for creativity and innovation,
allowing for the creation of new ideas and concepts.

2. Harmony and balance: Both mathematics and music rely on the concept of harmony and
balance to create a sense of beauty and order. In mathematics, this is achieved through
the use of symmetries, proportions, and the golden ratio, which create a sense of balance
and aesthetic appeal. Similarly, in music, harmony is created through the combination of
different notes and chords, whic

In [14]:
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

# 1m 40.3s 

 Llamas, Vicunas, and Alpacas are all South American camelids belonging to the same
family, Camelidae. They share similarities in appearance and behavior, but there are some
key differences between them.

1. Llamas: Llamas are the largest of the three species, with an average height of around 4
to 5 feet at the shoulder. They have long necks, large ears, and a distinctive humpless
back. Llamas are primarily used as pack animals and for their wool, which is coarser than
that of alpacas.

2. Vicunas: Vicunas are the smallest of the three species, with an average height of
around 3 to 4 feet at the shoulder. They have a slender build, long necks, and small ears.
Vicunas are known for their fine, soft wool, which is highly valued for its warmth and
softness. They are also considered endangered due to overhunting for their wool in the
past.

3. Alpacas: Alpacas are medium-sized camelids, with an average height of around 3 to 4
feet at the shoulder. They have a short, broad neck, small ears,

In [15]:
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)
         
# 1m 20.3s

 Subject: Open Sourcing GPT-4: A Valuable Contribution to the AI Community

Dear Sam,

I hope this email finds you well. As a strong advocate for the advancement of artificial
intelligence, I believe that OpenAI's GPT-4 is a significant milestone in the field.
However, I also recognize the potential benefits of making it open source.

Open sourcing GPT-4 would allow for increased collaboration, innovation, and transparency
within the AI community. This would enable researchers, developers, and enthusiasts to
contribute to the project, fostering a more diverse and inclusive environment.

Furthermore, open sourcing GPT-4 would encourage the development of new applications and
use cases, ultimately leading to a more comprehensive understanding of the technology's
capabilities. This could potentially lead to the discovery of new applications and use
cases that may not have been initially considered by OpenAI.

Additionally, open sourcing GPT-4 would promote responsible AI development and e

In [16]:
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)

# 1m 8.0s

 Dear Sam Altman,

I am Freddy, a 5-year-old boy who loves learning new things. I heard about GPT-4, a very
smart computer that can do many tasks. I think it would be a great idea to share GPT-4
with everyone by making it open source. Here are some reasons why:

1. Sharing is Caring: When we share toys with our friends, we all have more fun. In the
same way, if everyone can use GPT-4, it will help many people learn and create amazing
things.

2. Making Friends: GPT-4 can be like a friend who helps us with our homework, teaches us
new skills, and even makes us laugh. By sharing it, more people can have a friend like
GPT-4 to help them.

3. Helping the World: GPT-4 can do many things, like helping doctors find cures for
sicknesses or helping scientists discover new ways to protect our planet. By making it
open source, we can work together to make the world a better place.

I hope you consider my ideas, Sam. Let's make GPT-4 a friend for everyone!

Sincerely,
Freddy
CPU times: user 1min 8

In [17]:
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)

# 1m 23.6s

 Subject: Embracing Open Source GPT-4 for a Transparent and Inclusive Future

Dear Sam,

As the Vice President of the United States, I am writing to express my strong support for
the open-source release of GPT-4. This groundbreaking technology has the potential to
revolutionize various industries and improve the lives of countless individuals. However,
it is crucial to ensure that its development and implementation are guided by ethical
principles and promote transparency.

Open-sourcing GPT-4 would allow for a collaborative effort among researchers, developers,
and users, fostering innovation and accelerating progress. By making the code publicly
available, we can encourage a diverse range of experts to contribute their knowledge and
expertise, leading to better and more robust models. This approach would also help to
prevent regulatory capture, where powerful entities may attempt to manipulate the
technology for their own interests.

Moreover, open-sourcing GPT-4 would enable a more 

In [18]:
generate('What is the capital of England?',
         max_length=256)

# 2.0s

 The capital of England is London.
CPU times: user 2.02 s, sys: 2 µs, total: 2.02 s
Wall time: 2.02 s


In [19]:
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=256)

# 1m 4.5s

 To determine if it's possible for Geoffrey Hinton to have a conversation with George
Washington, we need to consider the time periods they lived in and the current state of
technology.

Geoffrey Hinton is a contemporary British-born computer scientist, known for his work in
deep learning and artificial intelligence. He was born in 1947.

George Washington was the first President of the United States, serving from 1789 to 1797.
He was born in 1732 and passed away in 1799.

Given the significant time difference between these two individuals, it's not possible for
them to have a direct conversation in the traditional sense. However, we can imagine a
conversation between them through the use of advanced technology, such as artificial
intelligence (AI) systems that can simulate the personalities and speech patterns of
historical figures.

In this case, a hypothetical conversation could be created by using AI to generate
responses based on the known thoughts, beliefs, and actions of both in

In [20]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         max_length=512)

# 2m 5.3s

 Once upon a time, in a quaint little town nestled among the Australian bush, there lived
a curious and talented koala named Koala-Bear. He was known for his exceptional skills in
various activities, from climbing trees to playing musical instruments. However, his most
remarkable talent was his ability to play pool.

One day, Koala-Bear decided to challenge the camelids, a group of camels, alpacas, and
llamas, who were known for their competitive spirit and love for games. They had heard
about Koala-Bear's prowess in pool and were eager to test their own skills against him.

The camelids gathered in the local pool hall, a cozy and welcoming place where they could
enjoy their favorite pastime. They were excited to face Koala-Bear, who had arrived with
his trusty cue stick, ready to showcase his skills.

As the game began, Koala-Bear carefully studied the table, analyzing the angles and
considering the best shots. He took his first shot, and the cue ball rolled smoothly
across the felt, 

## Chat

In [21]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

# 31.8s

 Try these tips to improve your focus while studying:

1. Create a dedicated study space: Designate a specific area for studying, free from
distractions. Make sure it's comfortable and well-lit.

2. Set a schedule: Plan your study sessions in advance, including breaks. Stick to your
schedule as much as possible.

3. Eliminate distractions: Turn off your phone, close unnecessary tabs on your computer,
and let others know you need quiet time to study.

4. Take breaks: Take short breaks (5-10 minutes) after studying for


## GSM8K

In [23]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

# 33.4s

 Step 1: Determine the number of apples left after lunch. The cafeteria initially had 23
apples and used 20 for lunch. So, 23 - 20 = 3 apples left after lunch.

Step 2: Add the 6 more apples that were bought. The cafeteria now has 3 apples left from
lunch + 6 new apples = 3 + 6 = 9 apples.

Step 3: Summarize the final number of apples. The cafeteria now has a total of 9 apples.


In [24]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=128)

# 32.7s

 First, we need to convert the 50 minutes of babysitting to hours. There are 60 minutes in
an hour, so 50 minutes is equal to 50/60 = 0.8333333333333333 hours. Now, we can calculate
her earnings by multiplying the hourly wage by the number of hours she worked: $12 per
hour * 0.8333333333333333 hours = $9.999999999999


In [25]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=256)

# 42.2s

 In the first hundred years, the monster consumed 847 people in total. Since ships have
been built larger over time, each new ship has twice as many people as the last ship. To
find the number of people on the ship in the first hundred years, we need to consider the
initial ship size.

Let's assume the initial ship had 1 person. After 100 years, the monster consumed 847
people. So, the total number of people on all ships in the first hundred years would be 1
(initial ship) + 847 (people consumed by the monster).

1 + 847 = 848 people

So, there were 848 people on the ship the monster ate in the first hundred years.


## CodeGen

In [26]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

# 47.0s

 def is_prime(number):
    if number <= 1:
        return False
    for i in range(2, int(number ** 0.5) + 1):
        if number % i == 0:
            return False
    return True


def print_prime(n):
    for i in range(2, n + 1):
        if is_prime(i):
            print(i)

print_prime(100)
```

Output:
2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97


In [27]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

# 12.2s

 def detect_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True
